In [1]:
!pip install xgboost==1.6.1
!pip install scikit-optimize


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 192.9 MB 70 kB/s 
  Attempting uninstall: xgboost
    Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 100 kB 8.1 MB/s 


In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

choose between xgbr, dtr and ann (1.rfr ,2.xgbr, 3.ann)

In [3]:
#from xgboost import XGBRegressor
#new_model=XGBRegressor()
#new_model.load_model('/content/drive/MyDrive/Colab Notebooks/xgbr_avggc.h5')

#pickle for rfr
import pickle
with open('/content/drive/MyDrive/Colab Notebooks/rfr_avggc.pkl' , 'rb') as f:
    new_model = pickle.load(f)

#from keras import models
# Recreate the exact same model, including its weights and the optimizer(r2/3)
#new_model = models.load_model('/content/drive/MyDrive/Colab Notebooks/my_tikamaveragegcr5-5r6.h5')
# Show the model architecture
#new_model.summary()

In [4]:
import pandas as pd
import numpy as np
test=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/average GC full dataset new for colab without powder mass.csv')
Xtest=test.iloc[:,:4].values
Ytest = test.iloc[:,4:].values
print(Xtest[0])


[2.00000000e+00 3.55247982e-01 3.00000000e+00 1.00000000e+03]


In [5]:
#prediction=new_model.predict(Xtest)
#print(prediction)
#print(Ytest)

#import numpy as np

#Defining MAPE function
#def MAPE(Y_actual,Y_Predicted):
 #   mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))*100
  #  return mape

#LR_MAPE= MAPE(Ytest,prediction)
#print("MAPE: ",LR_MAPE)
#print(Ytest[0,2])

In [6]:
#df=pd.DataFrame(prediction,index=None)
#arr=df[2]
#arr2=arr.to_numpy()
#print(arr2)

In [7]:
target=[(15,10,45)]
weight=[(0.5,0.1,1)]

def prediction(input):
   
   pred=new_model.predict(input)
   preddf=pd.DataFrame(pred,index=None)
   predvalue=preddf[2]
   predarr=predvalue.to_numpy()


   return predarr


In [8]:
def fitness(input):
    ans=prediction(input)
    return ans
  


In [9]:
target=[(15,10,45)]
weight=[(0.5,0.1,1)]
import random
#generate solutions
input=[]
for s in range(20):
    NPCtype=random.randint(1,2)
    
    #NPCweight=random.randrange(50000,61000,100)/1000
    loading=random.randrange(25,60,1)/100
    KOHconc=random.choice([1,3,0.75])
    #ratio=random.randrange(7,11,1)
    current=random.randrange(100,1001,100)


    input.append([(NPCtype,loading,KOHconc,current)])

input=np.asarray(input)
mating_pool=input.reshape(input.shape[0],input.shape[2])



In [10]:
test=prediction(mating_pool)
test=np.asarray(test)

print(mating_pool)
print(test)

[[1.0e+00 4.1e-01 1.0e+00 1.0e+02]
 [2.0e+00 4.5e-01 3.0e+00 5.0e+02]
 [1.0e+00 3.3e-01 7.5e-01 8.0e+02]
 [2.0e+00 5.8e-01 1.0e+00 8.0e+02]
 [1.0e+00 2.7e-01 1.0e+00 1.0e+03]
 [2.0e+00 3.4e-01 3.0e+00 2.0e+02]
 [1.0e+00 5.8e-01 7.5e-01 3.0e+02]
 [2.0e+00 5.1e-01 3.0e+00 9.0e+02]
 [1.0e+00 3.7e-01 3.0e+00 1.0e+03]
 [2.0e+00 3.9e-01 7.5e-01 6.0e+02]
 [1.0e+00 2.6e-01 1.0e+00 6.0e+02]
 [1.0e+00 4.6e-01 1.0e+00 4.0e+02]
 [1.0e+00 5.6e-01 1.0e+00 8.0e+02]
 [2.0e+00 3.7e-01 1.0e+00 4.0e+02]
 [1.0e+00 5.8e-01 3.0e+00 8.0e+02]
 [1.0e+00 5.7e-01 3.0e+00 3.0e+02]
 [2.0e+00 5.9e-01 3.0e+00 8.0e+02]
 [2.0e+00 5.4e-01 7.5e-01 9.0e+02]
 [1.0e+00 4.6e-01 3.0e+00 1.0e+03]
 [1.0e+00 2.7e-01 3.0e+00 1.0e+03]]
[20.38041933 35.63267613 40.75309863 35.13985225 27.51713032 23.1996761
 26.00965667 39.11448258 38.95690152 39.91662488 26.99409081 30.3874857
 34.92572278 26.51298287 38.6643893  25.97739855 39.121047   39.67170267
 40.14227369 29.16200158]


In [11]:
#select first four as parents(best 4)
def parent_selection(mixsort):
    fitness_mating_pool=prediction(mating_pool)
    mix_mating_fitness=np.column_stack((mating_pool,fitness_mating_pool))

    mixsort=mix_mating_fitness[mix_mating_fitness[:, 4].argsort()]
    parents_pool=mixsort[:,0:4]
    parents_results=mixsort[:,4]
    return parents_pool

In [12]:
# crossover(for mating)
def uniform_crossover(A,B,C,D,P):
    for i in range(len(P)):
        if P[i]<0.5:
          temp=A[i]
          A[i]=B[i]
          B[i]=C[i]
          C[i]=D[i]
          D[i]=temp
    return A,B,C,D

def crossover_process(mating_pool):
    A=mating_pool[0]
    B=mating_pool[1]
    C=mating_pool[2]
    D=mating_pool[3]
    P=np.random.rand(4)
    crossover_pool=uniform_crossover(A,B,C,D,P)
    crossover_pool=np.asarray(crossover_pool)
    return crossover_pool


#crossover_pool=crossover_process(parents_pool)
#print(crossover_pool)



In [13]:
#mutation!
def mutation(crossover_pool):
    mutation_rate=random.uniform(0.8,1.2)
    mutation_X=crossover_pool[:,0]
    mutation_pool=crossover_pool[:,1:]*mutation_rate
    final_mutation=np.column_stack((mutation_X,mutation_pool))
    return final_mutation

#mutation_pool=mutation(crossover_pool)
#print(mutation_pool)
#print(mutation_pool)
#mutation_rate=random.uniform(0.98,1.02)
#mutation_pool=crossover_pool*mutation_rate


In [14]:
#survival selection by def
def survivor_selection():
    new_mating_pool=np.insert(mutation(crossover_process(parent_selection(mating_pool))),0,mating_pool,axis=0)
    fitness_new_pool=fitness(new_mating_pool)
    mix_new_fitness=np.column_stack((new_mating_pool,fitness_new_pool))

    mixsortnew=mix_new_fitness[mix_new_fitness[:, 4].argsort()]
    mixsortnew =mixsortnew[::-1]
    mixsortnew=mixsortnew[:]
    new_mating_pool=mixsortnew[0:20,0:4]
    return new_mating_pool


test2=survivor_selection()
print(test2)

[[1.00000000e+00 3.30000000e-01 7.50000000e-01 8.00000000e+02]
 [1.00000000e+00 4.60000000e-01 3.00000000e+00 1.00000000e+03]
 [2.00000000e+00 3.90000000e-01 7.50000000e-01 6.00000000e+02]
 [2.00000000e+00 5.40000000e-01 7.50000000e-01 9.00000000e+02]
 [2.00000000e+00 5.90000000e-01 3.00000000e+00 8.00000000e+02]
 [2.00000000e+00 5.10000000e-01 3.00000000e+00 9.00000000e+02]
 [1.00000000e+00 3.70000000e-01 3.00000000e+00 1.00000000e+03]
 [1.00000000e+00 5.80000000e-01 3.00000000e+00 8.00000000e+02]
 [2.00000000e+00 4.50000000e-01 3.00000000e+00 5.00000000e+02]
 [2.00000000e+00 5.80000000e-01 1.00000000e+00 8.00000000e+02]
 [1.00000000e+00 5.60000000e-01 1.00000000e+00 8.00000000e+02]
 [1.00000000e+00 4.60000000e-01 1.00000000e+00 4.00000000e+02]
 [1.00000000e+00 2.70000000e-01 3.00000000e+00 1.00000000e+03]
 [1.00000000e+00 2.70000000e-01 1.00000000e+00 1.00000000e+03]
 [1.00000000e+00 2.60000000e-01 1.00000000e+00 6.00000000e+02]
 [2.00000000e+00 3.70000000e-01 1.00000000e+00 4.000000

In [15]:
def genetic_algorithm():
    parents=parent_selection(mating_pool)
    
    
    cross_p=crossover_process(parents)
   
    mutant_p=mutation(cross_p)
    survived_p=survivor_selection()
    return survived_p
    


print(genetic_algorithm())


[[1.00000000e+00 3.30000000e-01 7.50000000e-01 8.00000000e+02]
 [1.00000000e+00 4.60000000e-01 3.00000000e+00 1.00000000e+03]
 [2.00000000e+00 3.90000000e-01 7.50000000e-01 6.00000000e+02]
 [2.00000000e+00 5.40000000e-01 7.50000000e-01 9.00000000e+02]
 [2.00000000e+00 5.90000000e-01 3.00000000e+00 8.00000000e+02]
 [2.00000000e+00 5.10000000e-01 3.00000000e+00 9.00000000e+02]
 [1.00000000e+00 3.70000000e-01 3.00000000e+00 1.00000000e+03]
 [1.00000000e+00 5.80000000e-01 3.00000000e+00 8.00000000e+02]
 [2.00000000e+00 4.50000000e-01 3.00000000e+00 5.00000000e+02]
 [2.00000000e+00 5.80000000e-01 1.00000000e+00 8.00000000e+02]
 [1.00000000e+00 5.60000000e-01 1.00000000e+00 8.00000000e+02]
 [1.00000000e+00 4.60000000e-01 1.00000000e+00 4.00000000e+02]
 [1.00000000e+00 2.70000000e-01 3.00000000e+00 1.00000000e+03]
 [1.00000000e+00 2.70000000e-01 1.00000000e+00 1.00000000e+03]
 [1.00000000e+00 2.60000000e-01 1.00000000e+00 6.00000000e+02]
 [2.00000000e+00 3.70000000e-01 1.00000000e+00 4.000000

In [16]:
initial_gen=0
max_gen=1000



while initial_gen<max_gen:
   x_new=genetic_algorithm()
   initial_gen+=1 
   mating_pool=x_new
   
   first=x_new[0]
   first=np.asarray(first)
   print([first])
   print("gen",initial_gen,"=",x_new)
   
   test=new_model.predict(x_new)
   c2test=test[:,2]+test[:,4]
   print("best results:",test[0])
   
   print("Fittest C2H4+ethanol:",c2test[0])
   if c2test[0]>=75 : break
   

Streaming output truncated to the last 5000 lines.
 [2.00000000e+00 4.28908622e-01 2.55345810e+00 6.80922159e+02]
 [2.00000000e+00 4.05519420e-01 2.80526740e+00 5.79313457e+02]
 [2.00000000e+00 3.96224580e-01 3.67999547e+00 6.28798260e+02]
 [2.00000000e+00 4.02932765e-01 3.34816348e+00 6.24655772e+02]
 [2.00000000e+00 4.11619183e-01 2.74542315e+00 6.29415471e+02]
 [1.00000000e+00 3.30000000e-01 7.50000000e-01 8.00000000e+02]
 [2.00000000e+00 4.02160218e-01 3.18692890e+00 6.62209100e+02]
 [2.00000000e+00 3.87101273e-01 3.06392144e+00 6.83119894e+02]
 [1.00000000e+00 4.25023065e-01 2.77188955e+00 6.84361788e+02]]
best results: [13.02646964 19.70801784 42.60663046  1.60881015 16.98908022]
Fittest C2H4+ethanol: 59.59571068295009
[array([2.00000000e+00, 3.79289209e-01, 2.94460497e+00, 7.39791480e+02])]
gen 66 = [[2.00000000e+00 3.79289209e-01 2.94460497e+00 7.39791480e+02]
 [2.00000000e+00 3.79802875e-01 2.50985269e+00 7.72568633e+02]
 [2.00000000e+00 3.57925825e-01 9.31308514e-01 7.8948290

KeyboardInterrupt: ignored

In [17]:
print(test[0],mating_pool[0])
print(new_model.predict(mating_pool))


[11.75555674 22.89450597 43.25872255  1.53549896 15.55067825] [2.00000000e+00 3.66163197e-01 6.82868133e-01 7.28392675e+02]
[[11.75555674 22.89450597 43.25872255  1.53549896 15.55067825]
 [13.02646964 19.70801784 42.60663046  1.60881015 16.98908022]
 [13.02637726 19.70792862 42.60643243  1.60877895 16.98897402]
 [16.26397751 17.05703621 42.30904408  1.27681733 18.23936345]
 [13.60364944 20.80346035 41.37174253  1.44362144 15.82581331]
 [15.87854145 18.54123951 40.89509559  1.53024755 17.13624576]
 [12.44619476 22.16819695 40.89439129  2.11456897 16.59310239]
 [13.59841829 22.57610964 40.8865002   2.24731807 16.19852397]
 [13.59841829 22.57610964 40.8865002   2.24731807 16.19852397]
 [15.4239126  18.3619425  40.88455266  1.6126526  17.48337494]
 [15.41851719 18.31672933 40.86880671  1.60649268 17.47209427]
 [15.48552008 18.16889463 40.86256141  1.6135964  17.66311114]
 [15.48531764 18.17247654 40.86250266  1.6140685  17.66283964]
 [15.48531764 18.17247654 40.86250266  1.6140685  17.6628